In [1]:
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
from datetime import datetime
import re
import joblib
import sklearn

In [2]:
def StoreRdmSample():
    ## Load part of the training sample because the size is too large
    random.seed(4321)
    no_lines = 40428967
    size = 4 * 10**6
    skip = sorted(random.sample(range(1, no_lines), no_lines-size))
    train_df = pd.read_csv( "data/train.csv", header=0, skiprows=skip )
    train_df.to_csv("post_data/sampling.csv", index=False)

def AllUnique():
    f = open("data/train.csv",'r')
    line1 = f.readline()
    line1 = line1.strip('\n')
    header_list = re.split(',', line1)
    f.close()

    fp = open("feature_unique.txt", "a")
    fp2 = open("feature_No.txt", "a")
    for c in header_list:
        if c == "id": continue
        df = pd.read_csv("data/train.csv", usecols=[c])
        uni_list = df[c].unique()
        fp.write( c + ":" )
        fp.write( str(len(uni_list)) + "\n" )
        fp.write( str( uni_list ) )
        fp.write( "\n\n" )
        fp2.write( c + ":" )
        fp2.write( str(len(uni_list)) + "\n" )
    fp.close()
    fp2.close()
def SeperateTrainTest( df, num_train, _seed=9876 ):
    num_train_test = len(df)
    random.seed(_seed)
    shuffle_no_list = [x for x in range(num_train_test)]
    random.shuffle( shuffle_no_list ) # random.shuffle() has no return object
    train_no_list = shuffle_no_list[:num_train]
    test_no_list = shuffle_no_list[num_train:]
    train_df = df.iloc[train_no_list,:]
    test_df = df.iloc[test_no_list,:]
    train_df.index = range(len(train_df))
    test_df.index = range(len(test_df))
    
    return train_df, test_df

#StoreRdmSample()

### All are categories type, so to see the categories # of all variables
#AllUnique()

In [3]:
#sampling_df = pd.read_csv("post_data/sampling.csv")
#train_df, test_df = SeperateTrainTest(sampling_df, 2* 10**6)

#train_df.to_csv("post_data/train_sampling.csv", index=False)
#test_df.to_csv("post_data/test_sampling.csv", index=False)

#del sampling_df
"""
train_df = pd.read_csv("post_data/train_sampling.csv")
test_df = pd.read_csv("post_data/test_sampling.csv")
"""

'\ntrain_df = pd.read_csv("post_data/train_sampling.csv")\ntest_df = pd.read_csv("post_data/test_sampling.csv")\n'

In [4]:
def TransHour( df ):
    def EncodingHour( hour ):
    ### to 'hour' and 'weekday', 2 features
    ### Cut the hour in 8 range [1-4, 4-7, 7-10, 10-13, 13-16, 16-19, 19-22, 22-1]
        hour = str(hour)
        hour_str = "20" + hour[:6]
        day = datetime.strptime( hour_str, "%Y%m%d").weekday()
        day_hour = int(hour[6:])
        str_hour = None

        if day_hour in [1,2,3]: str_hour = str('1-4')
        elif day_hour in [4,5,6]: str_hour = str('4-7')
        elif day_hour in [7,8,9]: str_hour = str('7-10') 
        elif day_hour in [10,11,12]: str_hour = str('10-13') 
        elif day_hour in [13,14,15]: str_hour = str('13-16') 
        elif day_hour in [16,17,18]: str_hour = str('16-19') 
        elif day_hour in [19,20,21]: str_hour = str('19-22') 
        elif day_hour in [22,23,0]: str_hour = str('22-1')
        #print("day:", day)
        return day, str_hour
    
    weekday_list = ['weekday_0.0', 'weekday_1.0', 'weekday_2.0', 'weekday_3.0', 'weekday_4.0', 'weekday_5.0', 'weekday_6.0']
    dayhour_list = ['dayhour_1-4', 'dayhour_4-7', 'dayhour_7-10', 'dayhour_10-13', 'dayhour_13-16', 'dayhour_16-19', 'dayhour_19-22', 'dayhour_22-1']
    
    weekday_ser = pd.Series( np.zeros(shape=(len(df),)) )
    dayhour_ser = pd.Series( np.zeros(shape=(len(df),)) )
    for i in range(len(df)):
        weekday_ser[i], dayhour_ser[i] = EncodingHour(df.loc[i,'hour'])
        #print(weekday_ser[i])
        #print(dayhour_ser[i])
    df.loc[:,'weekday'] = weekday_ser
    df.loc[:,'dayhour'] = dayhour_ser   
    df['weekday'] = df['weekday'].astype('str')
    df['dayhour'] = df['dayhour'].astype('category')
    df = df.drop('hour', axis=1)
    
    df_dayhour_dum = pd.get_dummies( df['dayhour'], prefix="dayhour" )
    df_weekday_dum = pd.get_dummies( df['weekday'], prefix="weekday" )
    
    ## check if there are lack in weekday and dayhour
    for x in dayhour_list:
        if x not in df_dayhour_dum.columns:
            df_dayhour_dum.loc[:,x] = pd.Series( np.zeros(shape=(len(df_dayhour_dum),)) )
    for x in weekday_list:
        if x not in df_weekday_dum.columns:
            df_weekday_dum.loc[:,x] = pd.Series( np.zeros(shape=(len(df_weekday_dum),)) )
    ## re-order the dummies                                
    df_dayhour_dum = df_dayhour_dum[dayhour_list]
    df_weekday_dum = df_weekday_dum[weekday_list]
                                                
    df = pd.concat( [df, df_dayhour_dum], axis=1 )
    df = pd.concat( [df, df_weekday_dum], axis=1 )
    
    df.drop(['dayhour'], axis=1, inplace=True)
    df.drop(['weekday'], axis=1, inplace=True)
    
    return df

In [5]:
# Addressing hour first
#train_df.info()

#testing_train_df = train_df[:1000]
#testing_train_df = TransHour(testing_train_df)

#train_df = TransHour(train_df)
#test_df = TransHour(test_df)


In [6]:
#train_Cat_df, self_test_df = SeperateTrainTest(test_df, 10**6, 8765)

#train_Cat_df.to_csv("post_data/train_Cat_sampling.csv")
#self_test_df.to_csv("post_data/self_test_sampling.csv")
#del test_df
"""
train_Cat_df = pd.read_csv("post_data/train_Cat_sampling.csv")
self_test_df = pd.read_csv("post_data/self_test_sampling.csv")
"""

'\ntrain_Cat_df = pd.read_csv("post_data/train_Cat_sampling.csv")\nself_test_df = pd.read_csv("post_data/self_test_sampling.csv")\n'

In [7]:
# store time data respectively
"""
train_hour_data = train_df[['weekday_0.0', 'weekday_1.0', 'weekday_2.0', 'weekday_3.0', 'weekday_4.0', 'weekday_5.0', 'weekday_6.0', 'dayhour_1-4', 'dayhour_4-7', 'dayhour_7-10', 'dayhour_10-13', 'dayhour_13-16', 'dayhour_16-19', 'dayhour_19-22', 'dayhour_22-1']]
test_hour_data = self_test_df[['weekday_0.0', 'weekday_1.0', 'weekday_2.0', 'weekday_3.0', 'weekday_4.0', 'weekday_5.0', 'weekday_6.0', 'dayhour_1-4', 'dayhour_4-7', 'dayhour_7-10', 'dayhour_10-13', 'dayhour_13-16', 'dayhour_16-19', 'dayhour_19-22', 'dayhour_22-1']]
train_hour_data.to_csv("post_data/train_hour_data.csv", index=False)
test_hour_data.to_csv("post_data/test_hour_data.csv", index=False)
del train_hour_data
del test_hour_data
"""

'\ntrain_hour_data = train_df[[\'weekday_0.0\', \'weekday_1.0\', \'weekday_2.0\', \'weekday_3.0\', \'weekday_4.0\', \'weekday_5.0\', \'weekday_6.0\', \'dayhour_1-4\', \'dayhour_4-7\', \'dayhour_7-10\', \'dayhour_10-13\', \'dayhour_13-16\', \'dayhour_16-19\', \'dayhour_19-22\', \'dayhour_22-1\']]\ntest_hour_data = self_test_df[[\'weekday_0.0\', \'weekday_1.0\', \'weekday_2.0\', \'weekday_3.0\', \'weekday_4.0\', \'weekday_5.0\', \'weekday_6.0\', \'dayhour_1-4\', \'dayhour_4-7\', \'dayhour_7-10\', \'dayhour_10-13\', \'dayhour_13-16\', \'dayhour_16-19\', \'dayhour_19-22\', \'dayhour_22-1\']]\ntrain_hour_data.to_csv("post_data/train_hour_data.csv", index=False)\ntest_hour_data.to_csv("post_data/test_hour_data.csv", index=False)\ndel train_hour_data\ndel test_hour_data\n'

In [8]:
### classify vars to be lists
## A.  categories<10
## B.  100>categories>10 and with importance through human knowledge
## C.  10000>categories>100
## D.  categories>100000
A_list = ['C1', 'banner_pos', 'device_type', 'device_conn_type', 'C15', 'C16', 'C18']
B_list = ['site_category', 'app_category']
C_list = ['site_id', 'site_domain', 'app_id', 'app_domain', 'device_model', 'C14', 'C17', 'C19', 'C20', 'C21']
D_list = ['device_id', 'device_ip']

In [9]:
def MakeFtCatsDF( df, ft ):

    df_ct = pd.DataFrame(df[ft].value_counts())
    df_ct.columns = ["counts"]  # just 1 columns so far
    df_ct["id"] = df_ct.index	# col == 1
    df_ct["ratio"] = pd.Series( np.zeros(shape=(len(df_ct),)) ) # col == 2
    df_ct.index = range(len(df_ct))

    for x, i in zip(df_ct["id"], df_ct.index):
        tmp_df = df[df[ft]==x]
        No_nonclick = len(tmp_df[tmp_df["click"]==0])
        if No_nonclick == 0:
            ratio = 0.
        else:
            ratio = len(tmp_df[tmp_df["click"]==1])/No_nonclick
        df_ct.loc[i,"ratio"] = ratio
    return df_ct

def PlotFtCats( df, ft_list ):

    def PlotOneFt( df, ft ):
        df_ct = MakeFtCatsDF( df, ft )
        total_click = len(df[df["click"]==1])
        average_ratio = total_click/(len(df)-total_click)
        
        len_df_ct = len(df_ct)
        bin_list = range(0,len_df_ct,1)
        
        fig = plt.figure(figsize=(12,32), dpi=120)
        sb1 = fig.add_subplot(311)
        plt.hist( df_ct.index, weights=df_ct["counts"], color='grey', cumulative=False, alpha=0.5, bins=bin_list)
        plt.title( "Counts of category [" + ft + "]" )
        sb1.set_yscale('log')
        
        sb2 = fig.add_subplot(312)
        plt.hist( df_ct.index, weights=df_ct["ratio"], color='blue', cumulative=False, alpha=0.6, histtype='step', bins=bin_list)
        plt.axhline( y=average_ratio, color='r', linestyle='-')
        plt.title( "Click/Non-click ratio [" + ft + "]" )
        
        thirty_percent_cat = int(len(df_ct)/3)
        sb3 = fig.add_subplot(313)
        plt.hist( df_ct.index[:thirty_percent_cat], weights=df_ct["ratio"][:thirty_percent_cat], color='blue', cumulative=False, alpha=0.6, histtype='step', bins=bin_list[:thirty_percent_cat])
        plt.axhline( y=average_ratio, color='r', linestyle='-')
        plt.title( "Click/Non-click ratio (head-30% categories) [" + ft + "]" )
        sb3.set_yscale('log')

        plt.show()
        figname = "Fig/" + ft + "_Catscounts_CNratio.png"
        fig.savefig( figname )

    for ft in ft_list:
        PlotOneFt(df, ft)

## Deal with C type
#PlotFtCats( train_df, C_list )

In [10]:
import math
import json
def TransToOneFourth( ft, df ):
# [ "HIGH", "MIDHIGH", "MIDLOW", "LOW", "OTHER" ]
    
    df_ct = MakeFtCatsDF( df, ft )
    total_click = len(df[df["click"]==1])
    mean_ratio = total_click/(len(df)-total_click)
    sm_ratio = 0.
    for i in df_ct.index:
        sm_ratio += (df_ct.loc[i]["ratio"])**2 * df_ct.loc[i]["counts"]
    sm_ratio /= len(df)
    sigma_ratio = math.sqrt( sm_ratio - mean_ratio**2 )/math.sqrt(2.)

    print("Ft-" + ft +", mean_ratio: ", mean_ratio)
    print("Ft-" + ft +", sigma_ratio: ", sigma_ratio)

    df_high = df_ct[ df_ct["ratio"] >= (mean_ratio+sigma_ratio) ]
    df_midh = df_ct[ (df_ct["ratio"]<(mean_ratio+sigma_ratio)) & (df_ct["ratio"]>=mean_ratio) ]
    df_midl = df_ct[ (df_ct["ratio"]>=(mean_ratio-sigma_ratio)) & (df_ct["ratio"]<mean_ratio) ]
    df_low = df_ct[ df_ct["ratio"] < (mean_ratio-sigma_ratio) ]

    high_list = df_high["id"].to_list()
    midh_list = df_midh["id"].to_list()
    midl_list = df_midl["id"].to_list()
    low_list = df_low["id"].to_list()

    # Store for transforming the click-unknown testing sample 
    with open("Lists/HIGHlist_" + ft, "w") as f1:
        json.dump( high_list ,f1 )
    with open("Lists/MIDHIGHlist_" + ft, "w") as f2:
        json.dump( midh_list ,f2 )
    with open("Lists/MIDLOWlist_" + ft, "w") as f3:
        json.dump( midl_list ,f3 )
    with open("Lists/LOWlist_" + ft, "w") as f4:
        json.dump( low_list ,f4 )
    
    print("Ft-" + ft +", Finish storing 4 category lists!")

In [11]:
def DropNoUse(df):
    df.drop(["id"], axis=1, inplace=True)
    df.drop(drop_C_list, axis=1, inplace=True)
    df.drop(D_list, axis=1, inplace=True)
    return df

In [12]:
## By observation, I decide to drop the ['C17', 'C19', 'C20', 'C21']
drop_C_list = ['C17', 'C19', 'C20', 'C21']

C_list = [x for x in C_list if x not in drop_C_list]
print("New C_list:", C_list)

## for training
#for ft in C_list:
#    TransToOneFourth(ft, train_df)

New C_list: ['site_id', 'site_domain', 'app_id', 'app_domain', 'device_model', 'C14']


In [13]:
"""
train_df.info()
train_df = DropNoUse(train_df)
self_test_df = DropNoUse(self_test_df)
"""

'\ntrain_df.info()\ntrain_df = DropNoUse(train_df)\nself_test_df = DropNoUse(self_test_df)\n'

In [14]:
## Deal with B_list
## Do RF on the addition of the 2 'category' variables' one-hot encoding
## output should be the probability

def StoreRFTrain( df ):
    df = df[["site_category","app_category","click"]]
    df.info()
    RF_ft_train = pd.DataFrame(df['click'])
    for ft in B_list:
        df[ft] = df[ft].astype("category")
        dum_ft_train = pd.get_dummies( df[ft], prefix=ft )
        RF_ft_train = pd.concat([RF_ft_train, dum_ft_train], axis=1)
    RF_ft_train.info()
    ## store the training feature for later applying
    print("RF_ft_train.columns", RF_ft_train.columns)
    with open("Lists/RF_fts.txt", "w") as f:
        json.dump( RF_ft_train.columns.to_list() ,f ) # the 1st column is 'click'
    
    RF_ft_train.to_csv("post_data/dum_RF_train.csv", index=False)
    print( "Finish Storing dum training info for RF!")

def RFCatTrain( N_est=200, crit='gini', rdmstate=1, nj=-1 ):
    dum_feature = pd.read_csv("post_data/dum_RF_train.csv")
    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier(n_estimators=N_est, criterion=crit, random_state=rdmstate, n_jobs=nj)
    forest.fit( dum_feature.iloc[:,1:], dum_feature['click'] ) # columns index=0 is 'click'
    print( "Finish RF fit!")
    
    model_name = "Model/RFCat_" + str(N_est) + "_" + crit + "_" + str(rdmstate) + "_" + str(nj) 
    joblib.dump( forest, model_name )
    print("Finish training RF Cat!")

In [15]:
## seperate half test_df to be the training sample for RF
## for training!
#train_Cat_df.info()
#StoreRFTrain(train_Cat_df)
#RFCatTrain()

In [16]:
# to apply OneFourth on other df
def OneFourth_Ft( ft, df ):

    high_list = []
    midh_list = []
    midl_list = []
    low_list = [] 

    with open("Lists/HIGHlist_" + ft, "r") as f1:
        high_list = json.load( f1 )
    with open("Lists/MIDHIGHlist_" + ft, "r") as f2:
        midh_list = json.load( f2 )
    with open("Lists/MIDLOWlist_" + ft, "r") as f3:
        midl_list = json.load( f3 )
    with open("Lists/LOWlist_" + ft, "r") as f4:
        low_list = json.load( f4 )

    for x in high_list:
        df.loc[ df[ft]==x, ft ] = "HIGH"
    for x in midh_list:
        df.loc[ df[ft]==x, ft ] = "MIDHIGH"
    for x in midl_list:
        df.loc[ df[ft]==x, ft ] = "MIDLOW"
    for x in low_list:
        df.loc[ df[ft]==x, ft ] = "LOW"
    df.loc[ (df[ft]!="HIGH")&(df[ft]!="MIDHIGH")&(df[ft]!="MIDLOW")&(df[ft]!="LOW"), ft ] = "OTHER"

    dum_ft_df = pd.get_dummies(df[ft], prefix=ft)
    if ft+"_OTHER" in dum_ft_df.columns:
        dum_ft_df.drop([ft+"_OTHER"], axis=1, inplace=True) # "OTHER" would appear only in testing sample, not in training sample
    df.drop([ft], axis=1, inplace=True)
    df = pd.concat([df, dum_ft_df], axis=1)
    
    print("Finish dealing with feature > " + ft + "!")

    return df

# to apply the trained RF on other df
def RF_Ft(df):
    df_dum_site = pd.get_dummies( df['site_category'], prefix='site_category' )
    df_dum_app = pd.get_dummies( df['app_category'], prefix='app_category' )
    df_dum = pd.concat([df_dum_site, df_dum_app], axis=1)
    
    main_col_list = []
    with open("Lists/RF_fts.txt", "r") as f:
        main_col_list = json.load( f )
    ## main_col_list[0] == "click"
    main_col_list = main_col_list[1:]

    input_list = df_dum.columns.to_list()
    for x in main_col_list:
        if x not in input_list:
            df_dum[x] = pd.Series( np.zeros(shape=(len(df_dum),)) )

    df_dum = df_dum[main_col_list]
    print(main_col_list)
    print(df_dum.columns.to_list())
    
    forest = joblib.load( "Model/RFCat_200_gini_1_-1" )
    rf_ser = forest.predict( df_dum )
    df["RFCat"] = rf_ser
    df.drop(B_list, axis=1, inplace=True)
    
    print("Finish transforming RF!")

    return df

def TransOneHotA_train(df): # for training
    A_fts_list = []
    dum_df = pd.DataFrame()
    for ft in A_list:
        dum_ft = pd.get_dummies( df[ft], prefix=ft )
        #dum_ele = dum_ft.columns.to_list()
        #A_fts_list = [*A_fts_list, *dum_ele]
        df.drop([ft], axis=1, inplace=True)
        dum_df = pd.concat([dum_df, dum_ft], axis=1)
    df = pd.concat([df, dum_df], axis=1)
    with open("Lists/A_fts.txt", "w") as f:
        json.dump( dum_df.columns.to_list() ,f )
    print("Finish One-Hot A!")

    return df

def TransOneHotA(df): # for testing 
    with open("Lists/A_fts.txt", "r") as f:
        A_fts_list = json.dump( f )

    dum_df = pd.DataFrame()
    for ft in A_list:    
        dum_ft = pd.get_dummies( df[ft], prefix=ft )
        df.drop([ft], axis=1, inplace=True)
        dum_df = pd.concat([dum_df, dum_ft], axis=1)
        
    for x in A_fts_list:
        if x not in df:
            dum_df[x] = pd.Series( np.zeros(shape=(len(df_dum),)) )
    
    dum_df = dum_df[A_fts_list]
    df = pd.concat([df, dum_df], axis=1)
    
    print("Finish One-Hot A!")
    
    return df

In [17]:
#testing_train_df = train_df[:1000]
#testing_train_df = TransOneHotA_train(testing_train_df)
#for ft in C_list:
#    testing_train_df = OneFourth_Ft(ft, testing_train_df)
#testing_train_df = RF_Ft(testing_train_df)
#testing_train_df.info()
#print(testing_train_df.head(20))

In [18]:
"""
train_df = TransOneHotA_train(train_df)
for ft in C_list:
    train_df = OneFourth_Ft(ft, train_df)
train_df = RF_Ft(train_df)
train_df.info()
"""

'\ntrain_df = TransOneHotA_train(train_df)\nfor ft in C_list:\n    train_df = OneFourth_Ft(ft, train_df)\ntrain_df = RF_Ft(train_df)\ntrain_df.info()\n'

In [19]:
#train_df.to_csv("post_data/train_Ft.csv")
#train_df.info()

In [20]:
"""
self_test_df = TransOneHotA(self_test_df)
for ft in C_list:
    self_test_df = OneFourth_Ft(ft, self_test_df)
self_test_df = RF_Ft(self_test_df)
self_test_df.info()
self_test_df.to_csv("post_data/self_test_Ft.csv")
"""

'\nself_test_df = TransOneHotA(self_test_df)\nfor ft in C_list:\n    self_test_df = OneFourth_Ft(ft, self_test_df)\nself_test_df = RF_Ft(self_test_df)\nself_test_df.info()\nself_test_df.to_csv("post_data/self_test_Ft.csv")\n'

In [21]:
# deal with the final dealing df
print(">>>>> step", 0 )
value_df = pd.read_csv("data/test.csv")
print(">>>>> step", 1 )
value_df = TransHour(value_df)
print(">>>>> step", 2 )
time_list = ['weekday_0.0', 'weekday_1.0', 'weekday_2.0', 'weekday_3.0', 'weekday_4.0', 'weekday_5.0', 'weekday_6.0', 'dayhour_1-4', 'dayhour_4-7', 'dayhour_7-10', 'dayhour_10-13', 'dayhour_13-16', 'dayhour_16-19', 'dayhour_19-22', 'dayhour_22-1']
value_hour_data = value_df[time_list]
print(">>>>> step", 3 )

>>>>> step 0
>>>>> step 1
>>>>> step 2
>>>>> step 3


In [22]:
value_df.drop(time_list, axis=1, inplace=True)
print(">>>>> step", 4 )
value_df = DropNoUse(value_df)
print(">>>>> step", 5 )
value_df = TransOneHotA(value_df)
print(">>>>> step", 6 )
for ft in C_list:
    value_df = OneFourth_Ft(ft, value_df)
print(">>>>> step", 7 )
value_df = RF_Ft(value_df)
print(">>>>> step", 8 )
value_df = pd.concat([value_df, value_hour_data], axis=1)
print(">>>>> step", 9 )
value_df.to_csv("post_data/value_Ft.csv")
print(">>>>> step", 10 )
value_df.info()

>>>>> step 4
>>>>> step 5
Finish One-Hot A!
>>>>> step 6
Finish dealing with feature > site_id!
Finish dealing with feature > site_domain!
Finish dealing with feature > app_id!
Finish dealing with feature > app_domain!
Finish dealing with feature > device_model!
Finish dealing with feature > C14!
>>>>> step 7
['site_category_0569f928', 'site_category_28905ebd', 'site_category_335d28a8', 'site_category_3e814130', 'site_category_42a36e14', 'site_category_50e219e0', 'site_category_5378d028', 'site_category_70fb0e29', 'site_category_72722551', 'site_category_75fa27f6', 'site_category_76b2941d', 'site_category_8fd0aea4', 'site_category_9ccfa2ea', 'site_category_a818d37a', 'site_category_bcf865d9', 'site_category_c0dd3be3', 'site_category_dedf689d', 'site_category_e787de0e', 'site_category_f028772b', 'site_category_f66779e6', 'app_category_07d7df22', 'app_category_09481d60', 'app_category_0bfbc358', 'app_category_0d82db25', 'app_category_0f2161f8', 'app_category_0f9a328c', 'app_category_18b1